In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from astropy.coordinates import SkyCoord as coord
import astropy.units as u
from astropy.coordinates import Angle
import warnings
warnings.filterwarnings('ignore')

###  Step 1:  Importing gamma ray burst data

<font color = 'purple'>Data I have used for gamma ray burst was taken form this site below.</font>

https://swift.gsfc.nasa.gov/results/batgrbcat/summary_cflux/summary_general_info/summary_general.txt


In [2]:
column_names = ['GRBname', 'RA_ground', 'DEC_ground']
df_grb = pd.read_csv('/home/astroguy/frb_summerresearch/grb_data_copy.csv', delimiter='|', header = 0)
df_grb.head(1)

,GRBname,Trig_ID,Trig_time_met,Trig_time_UTC,RA_ground,DEC_ground,Image_position_err,Image_SNR,T90,T90_err,T50,T50_err,Evt_start_sincetrig,Evt_stop_sincetrig,pcode,Trigger_method,XRT_detection,"comment,,,"
0,GRB200829A,993768,620402400.000,2020-08-29T13:59:34.457300,2.511393e+02,7.236396e+01,1.821205e+00,1.291746e+01,1.310000e+01,3.079558e+00,2.684000e+00,4.882629e-02,-239,963,0.8828,image trigger,Yes,",,,"


<font color = 'purple'>Note:  According to the description in the catalog RA_ground and DEC_ground both has been measured in the unit of degree.</font> 

In [58]:
df_grb_subset = df_grb.iloc[:,[0,4,5]]
df_grb_subset.tail(5)

,GRBname,RA_ground,DEC_ground
1386,GRB041220,2.913007e+02,6.059630e+01
1387,GRB041219C,3.438824e+02,-7.678556e+01
1388,GRB041219B,1.676888e+02,-3.346167e+01
1389,GRB041219A,N/A,N/A
1390,GRB041217,1.647929e+02,-1.794047e+01


In [62]:
df_grb_subset.iloc[1388][2]

' -3.346167e+01 '

In [42]:
float(df_grb_subset.iloc[0][2])

72.36396

###  Step 2: Working with fast radio burst data

<font color = 'purple'>Data taken from link below</font>

https://www.herta-experiment.org/frbstats/catalogue

In [4]:
columns = ['ra', 'dec', 'dm', 'flux', 'redshift', 'frb', 'l', 'b', 'width', 'fluence']
df_herta = pd.read_csv('herta_frb_catalogue.csv', usecols=columns).replace('-', '0')
df_herta.head(1)

,frb,ra,dec,l,b,dm,flux,width,fluence,redshift
0,FRB 20010125A,19:06:53,-40:37:14,356.64,-20.02,790.3,0.54,0,0,0.7083


<font color = 'purple'>Unit of RA in the catalog is hour:min:sec and that of declination is in deg:arcmin:arcsec</font>

In [5]:
herta_subset = df_herta.iloc[:, [0, 1,2]]
herta_subset.head(1)

,frb,ra,dec
0,FRB 20010125A,19:06:53,-40:37:14


<font color = 'purple'>We convert RA and DEC to the unit of degree to make it uniform with the unit of RA and DEC in GRB data. We use astropy library to convert coordinates to degress. </font>

Reference:  https://docs.astropy.org/en/stable/coordinates/angles.html

<font color = 'purple'>This site maintained by penstate can be used as checker to see if our conversion has been done correctly.</font>

https://www.swift.psu.edu/secure/toop/convert.htm

In [29]:
RA_in_degrees  = []
DEC_in_degrees = []

for k in np.arange(0, len(herta_subset['frb'])):
    RA_degree = Angle(herta_subset['ra'][k], unit = 'hourangle')   # RA is in  hr:min:sec
    DEC_degree = Angle(herta_subset['dec'][k], unit = 'degree')    # DEC is in deg:arcmin:arcsec
    RA_in_degrees.append(RA_degree.degree)
    DEC_in_degrees.append(DEC_degree.degree)
    
herta_subset['RA_in_deg'] = RA_in_degrees
herta_subset['DEC_in_deg'] = DEC_in_degrees

herta_subset.head(5)

,frb,ra,dec,RA_in_deg,DEC_in_deg
0,FRB 20010125A,19:06:53,-40:37:14,286.720833,-40.620556
1,FRB 20010305A,4:57:20,-52:36:25,74.333333,-52.606944
2,FRB 20010312A,5:26:55,-64:56:19,81.729167,-64.938611
3,FRB 20010621A,18:52:05,-8:29:35,283.020833,-8.493056
4,FRB 20010724A,1:18:06,-75:12:18,19.525000,-75.205000


In [83]:
def distance(ra_frb, ra_grb, dec_frb, dec_grb):
    diff_ra = ra_frb - ra_grb
    diff_dec = dec_frb - dec_grb
    avg_dec = (dec_frb + dec_grb)/2
    frb_grb_dist = np.sqrt((diff_ra*np.cos(avg_dec*(np.pi/180)))**2 + diff_dec**2)
    return frb_grb_dist

In [93]:
dist_s_no = []
for m in np.arange(0, len(herta_subset['frb'])-800):
    frb_ra = herta_subset['RA_in_deg'][m]
    frb_dec = herta_subset['DEC_in_deg'][m]

    dist_list = []
    for n in np.arange(0, len(df_grb_subset.iloc[:])-1380):
        grb_ra = float(df_grb_subset.iloc[n][1])
        grb_dec = float(df_grb_subset.iloc[n][2])
        
        dist = distance(frb_ra, grb_ra, frb_dec, grb_dec)
        dist_list.append(dist)
        
    min_dist_val = min(dist_list)
    grb_index = dist_list.index(min_dist_val)
    dist_s_no.append([m, grb_index, min_dist_val])
    
print(dist_s_no)

[[0, 10, 12.612198162327877], [1, 9, 12.424373352043105], [2, 9, 6.056852305815897], [3, 7, 8.861887449747282], [4, 2, 1.6830365965367302], [5, 6, 25.765247573371585]]


In [82]:
emp = []

for k in np.arange(0, 5):
    f = k**2
    emp.append(f)

print(emp)
max(emp)
emp.index(16)

[0, 1, 4, 9, 16]


4

In [90]:
len(herta_subset['frb']-800)

TypeError: unsupported operand type(s) for -: 'str' and 'int'

In [17]:
np.sqrt(25)

5.0

In [87]:
type(herta_subset['RA_in_deg'][0])

numpy.float64